In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import torch as tc
import torchvision 
from torchvision import datasets
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from dlc_practical_prologue import *

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-03-23 23:14:26--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-23 23:14:26--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.9’

MNIST.tar.gz.9          [    <=>             ]  33.20M  2.70MB/s    in 13s     

2021-03-23 23:14:40 (2.58 MB/s) - ‘MNIST.tar.gz.9’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

In [ ]:
transforms = torchvision.transforms.ToTensor()
train_data = datasets.MNIST(root = './', train=True, download=True)
val_data = datasets.MNIST(root = './', train=False, download=True)

In [ ]:
train_input = train_data.data.view(-1, 1, 28, 28).float()
train_target = train_data.targets
test_input = val_data.data.view(-1, 1, 28, 28).float()
test_target = val_data.targets

In [ ]:
train_input = train_input.clone().reshape(train_input.size(0), -1)
test_input = test_input.clone().reshape(test_input.size(0), -1)

In [ ]:
print('** Reduce the data-set to the tiny setup')
train_input = train_input.narrow(0, 0, 500)
train_target = train_target.narrow(0, 0, 500)
test_input = test_input.narrow(0, 0, 100)
test_target = test_target.narrow(0, 0, 100)

** Reduce the data-set to the tiny setup


In [ ]:
train_target = convert_to_one_hot_labels(train_input, train_target)
test_target = convert_to_one_hot_labels(test_input, test_target)

In [ ]:
mu, std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)
test_input.sub_(mu).div_(std)

tensor([[-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        ...,
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167],
        [-0.4167, -0.4167, -0.4167,  ..., -0.4167, -0.4167, -0.4167]])

In [ ]:
train_target = 0.9 * train_target
test_target = 0.9 * test_target

In [ ]:
train_target.shape

torch.Size([500, 10])

#Question 1 Activation function

In [ ]:
def sigma(x):
  return tc.tanh(x)

In [ ]:
def dsigma(x):
  return 1-(tc.tanh(x))**2

#Question 2 Loss functions

In [ ]:
def loss(v,t):
  return tc.sum(tc.pow((t-v),2))
  


In [ ]:
def dloss(v,t):
  return 2*(t-v)


#Question 3  Forward and backward passes

In [ ]:
def forward_pass(w1, b1, w2, b2, x):
  x0 = x
  s1 = tc.mm(w1,x0)+b1
  x1 = sigma(s1)
  s2 = tc.mm(w2,x1)+b2
  x2 = sigma(s2)
  return x0,s1,x1,s2,x2


In [ ]:
def backward_pass(w1,b1,w2, b2,t,x,s1, x1,s2,x2,dl_dw1,dl_db1, dl_dw2, dl_db2):
  #dl_dw2 += tc.mm(tc.mul(dloss(x2,t),dsigma(s2)), tc.t(x1))
  #dl_dw1 += tc.mm(tc.mul(tc.mm(tc.t(w2),tc.mul( dloss(x2,t), dsigma(s2))), dsigma(s1)), tc.t(x))
  #dl_db2 += tc.mul(dloss(x2,t),dsigma(s2))
  #dl_db1 += tc.mul(tc.mm(tc.t(w2), tc.mul(dloss(x2,t),dsigma(s2))), dsigma(s1))
  ########################
  dl_dx2 = dloss(x2, t)
  dl_ds2 = tc.mul(dl_dx2, dsigma(s2))
  dl_dw2 += tc.mm(dl_ds2, tc.t(x1))
  dl_db2 += dl_ds2
  dl_dx1 = tc.mm(torch.t(w2), dl_ds2)
  dl_ds1 = tc.mul(dl_dx1, dsigma(s1))
  dl_dw1 += tc.mm(dl_ds1, tc.t(x))
  dl_db1 += dl_ds1

  #return None
  

  


#Question 4  Training the network

In [ ]:
# Compute the error
def error(test_set, test_target_set, w1, b1, w2, b2):
  n = test_set.shape[0]
  num_errors = 0
  for i in range(n):
    x = x = train_input [i].view(-1,1)
    t = torch.argmax(test_target_set[i])
    _, _, _, _, predicted = forward_pass(w1, b1, w2, b2, x)
    predicted_class = torch.argmax(predicted, 0)
    if t != predicted_class:
      num_errors += 1
  return num_errors/n

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
def train(train_input, train_target, test_input, test_target):

  lr = 0.1/train_target.shape[0]
  eps=1e-6
  m,n= train_input.shape

  # set weights
  w2 = tc.empty(10, 50).normal_(0, eps)
  b2 = tc.empty(10, 1).normal_(0, eps)
  w1 = tc.empty(50, 784).normal_(0, eps)
  b1 = tc.empty(50, 1).normal_(0, eps)
# set gradients
  dl_dw2 = tc.zeros(10, 50)
  dl_db2 = tc.zeros(10, 1)
  dl_dw1 = tc.zeros(50, 784)
  dl_db1 = tc.zeros(50, 1)
  for u in range(1000):

    for k in range(m):

      x = train_input [k].view(-1,1)
      t = train_target[k].view(-1,1)
      x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, x)
      backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2)
    
    w2 = w2 - lr * dl_dw2
    b2 = b2 - lr * dl_db2
    w1 = w1 - lr * dl_dw1
    b1 = b1 - lr * dl_db1
  
  
  Training_error = error(train_input,train_target, w1, b1, w2, b2)
  print("Training_error is {}".format(Training_error*100,".1f") + "%")

  Test_error = error(test_input,test_target, w1, b1, w2, b2)
  print("Test_error is {}".format(Test_error*100,".1f") + "%")
  return w1, w2, b1, b2, x2

  

In [ ]:
w1, w2, b1, b2, pred= train(train_input, train_target, test_input, test_target)

Training_error is 90.0%
Test_error is 92.0%


In [ ]:
print(accuracy(pred, test_target)) 

tensor(0.9000)


In [ ]:
w1

tensor([[ 1.8705,  1.8705,  1.8705,  ...,  1.8705,  1.8705,  1.8705],
        [-2.1346, -2.1346, -2.1346,  ..., -2.1346, -2.1346, -2.1346],
        [ 0.2530,  0.2530,  0.2530,  ...,  0.2530,  0.2530,  0.2530],
        ...,
        [-1.1047, -1.1047, -1.1046,  ..., -1.1046, -1.1047, -1.1046],
        [ 2.4444,  2.4444,  2.4444,  ...,  2.4444,  2.4444,  2.4444],
        [-0.3839, -0.3839, -0.3839,  ..., -0.3839, -0.3839, -0.3839]])

In [ ]:
w2

tensor([[ 23.8390, -28.3390,   2.7956, -25.4600, -22.0105,  25.4197,  -2.7683,
          -7.2523,  19.4511,  -4.5157,  29.3693, -17.7986, -45.6945, -28.8914,
         -17.4993, -14.7104,  14.8721,  19.6195, -25.3767,  34.0264,  -6.9812,
          -9.6228,  -6.4847, -14.7161, -10.2590,  -9.7599, -17.1122,  23.6342,
           5.1321,  12.2356,  38.8129,  16.3011,  26.9367, -35.1590,   3.0091,
         -29.2596,  22.6371,  45.8154,  -4.1285,  -7.0543,   3.8797, -33.9190,
         -11.6893,  -5.5913,  39.2748,  -1.5500,   9.0265, -12.9631,  34.4391,
          -4.2719],
        [ 17.4690, -20.6860,   2.0802, -18.6258, -16.1381,  18.5720,  -2.0753,
          -5.3783,  14.2886,  -3.3151,  21.4251, -13.0801, -33.4929, -21.1323,
         -12.8653, -10.8272,  10.9749,  14.4311, -18.5542,  24.8768,  -5.1921,
          -7.1215,  -4.8275, -10.8550,  -7.6269,  -7.2498, -12.5832,  17.2828,
           3.8093,   9.0190,  28.3703,  12.0201,  19.6911, -25.6759,   2.2597,
         -21.3741,  16.5939,  33

In [ ]:
b1.shape

torch.Size([50, 1])

In [ ]:
b2.shape

torch.Size([10, 1])

In [ ]:
pred.shape

torch.Size([10, 1])